In [1]:
import numpy as np
import pandas as pd

In [2]:
amb_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/Utilization/amb-final-county.csv')
tele_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/q3014_county_final.csv')

In [3]:
# Counties that exist at least once (in the full data set w/ all counties in the US)
all_c = sorted(amb_df.fips.value_counts().index.tolist())
# All years
all_y = [2013,2014,2015,2016,2017,2018,2019]
# Together
df = pd.concat([pd.DataFrame(data=np.repeat(np.array(all_c), repeats=len(all_y)), columns=['fips']),
                pd.DataFrame(data=np.tile(np.array(all_y), reps=len(all_c)), columns=['year'])], axis=1)

In [4]:
# Whether a county implemented telehealth in that year
new_list = []
for i in range(len(df)):
    try:
        if len(tele_df[(tele_df['year']==df['year'][i])&(tele_df['fips']==df['fips'][i])])==0:
            new_list.append(0)
        elif len(tele_df[(tele_df['year']==df['year'][i])&(tele_df['fips']==df['fips'][i])])!=0:
            new_list.append(1)
    except:
        print(i)
df['tele']=new_list

In [5]:
# These counties did not adopt telehealth; they can be used as the control group
cg_df = df.groupby(['fips']).sum().reset_index(level=[0])
cg_list = cg_df[cg_df['tele']==0].fips.values.tolist()


In [6]:
# These counties adopted telehealth 
tg_df = df[(~df['fips'].isin(cg_list))&(df['tele']==1)].groupby(['fips']).min().reset_index(level=[0])
# Let's see when they each begin to implement telehealth
tg_dict = {}
for i in range(len(tg_df)):
    tg_dict[tg_df['fips'][i]] = tg_df['year'][i]

In [42]:
tg_df.groupby(['year']).size()
# We may better use 2015; when there were 103 new adopters
# Or 2016; when there weree 119 new adopters

year
2013    198
2014     89
2015    103
2016    119
2017    111
2018    101
2019     92
dtype: int64

##### 0. DID without matching

0.1 Counties with telehealth since >=2015 vs Control group

In [7]:
tg_list = tg_df[tg_df['year']>=2015].fips.values.tolist()

In [8]:
len(tg_list), len(cg_list)

(526, 2310)

In [9]:
df_0 = df[df['fips'].isin(tg_list+cg_list)].reset_index(drop=True)
df_0['tg'] = [1 if i in tg_list else 0 for i in df_0['fips']]
temp_df = pd.merge(df_0, tele_df[['fips','year','tot_srvcs','tot_srvcs_chrg']], on=['fips','year'], how='outer')
temp_df = temp_df[~temp_df['tele'].isna()].reset_index(drop=True)
temp_df = temp_df.fillna(0)


In [10]:
temp_df.info()
df_0 = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19852 entries, 0 to 19851
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fips            19852 non-null  int64  
 1   year            19852 non-null  int64  
 2   tele            19852 non-null  float64
 3   tg              19852 non-null  float64
 4   tot_srvcs       19852 non-null  float64
 5   tot_srvcs_chrg  19852 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 930.7 KB


In [11]:
temp_df = pd.merge(df_0, amb_df, on=['fips','year'], how='outer')
temp_df = temp_df[~temp_df['tele'].isna()].reset_index(drop=True)
temp_df = temp_df[~temp_df['reference_period'].isna()].reset_index(drop=True)

In [12]:
temp_df.info()
df_0 = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14075 entries, 0 to 14074
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   fips                                     14075 non-null  int64  
 1   year                                     14075 non-null  float64
 2   tele                                     14075 non-null  float64
 3   tg                                       14075 non-null  float64
 4   tot_srvcs                                14075 non-null  float64
 5   tot_srvcs_chrg                           14075 non-null  float64
 6   reference_period                         14075 non-null  object 
 7   state                                    14075 non-null  object 
 8   county                                   14075 non-null  object 
 9   state_fips                               14075 non-null  float64
 10  county_fips                              14075

In [26]:
df_0.to_csv(r'did_tele_ambulatory_nomatching_county_2015.csv', index=False)

In [14]:
# fips to HSA
hsa_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/hsatofips/fipstohsa.csv')
hsa_df.columns=['hsa','hsa_desc','state_county_desc','fips']

In [23]:
temp_df = pd.merge(df_0, hsa_df, on=['fips'], how='outer')
temp_df = temp_df[~temp_df['hsa'].isna()].reset_index(drop=True) # Mostly AK and SD
df_0 = temp_df.copy()

In [72]:
df_0_tele_gb = df_0.groupby(['hsa','year']).sum().reset_index(level=[0,1])[['hsa','year','tot_srvcs','tot_srvcs_chrg',
                                                    'number_of_fee_for_service_beneficiaries', 'number_of_providers_allamb',
                                                    'number_of_users_allamb', 'total_payment_allamb',
                                                    'number_of_users_emeramb', 'total_payment_emeramb',
                                                    'number_of_providers_emeramb', 'number_of_users_nonemeramb','total_payment_nonemeramb', 'number_of_providers_nonemeramb']]
df_0_tele_gb = df_0_tele_gb.sort_values(by=['hsa','year']).reset_index(drop=True)
df_0_date_gb = df_0.groupby(['hsa','year']).max().reset_index(level=[0,1])[['hsa','year','tele','tg']]
df_0_gb = pd.merge(df_0_date_gb, df_0_tele_gb, on=['hsa','year'])

In [85]:
temp_df = pd.merge(df_0[['hsa','state_county_desc']], df_0_gb, on=['hsa'], how='outer')
temp_df = temp_df[~temp_df['tele'].isna()].reset_index(drop=True)
temp_df['state'] = [str(i).split(':')[0] for i in temp_df['state_county_desc']]
df_0 = temp_df.copy()

In [92]:
df_0 = pd.merge(df_0, amb_df[['state','state_fips']].drop_duplicates(), on=['state'])

In [93]:
df_0.to_csv(r'did_tele_ambulatory_nomatching_hsa_2015.csv', index=False)

0.2 Counties with telehealth since >=2016 vs Control group

In [8]:
tg_list = tg_df[tg_df['year']>=2016].fips.values.tolist()

In [9]:
len(tg_list), len(cg_list)

(423, 2310)

In [10]:
df_1 = df[df['fips'].isin(tg_list+cg_list)].reset_index(drop=True)
df_1['tg'] = [1 if i in tg_list else 0 for i in df_1['fips']]
temp_df = pd.merge(df_1, tele_df[['fips','year','tot_srvcs','tot_srvcs_chrg']], on=['fips','year'], how='outer')
temp_df = temp_df[~temp_df['tele'].isna()].reset_index(drop=True)
temp_df = temp_df.fillna(0)


In [11]:
temp_df.info()
df_1 = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19131 entries, 0 to 19130
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fips            19131 non-null  int64  
 1   year            19131 non-null  int64  
 2   tele            19131 non-null  float64
 3   tg              19131 non-null  float64
 4   tot_srvcs       19131 non-null  float64
 5   tot_srvcs_chrg  19131 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 896.9 KB


In [12]:
temp_df = pd.merge(df_1, amb_df, on=['fips','year'], how='outer')
temp_df = temp_df[~temp_df['tele'].isna()].reset_index(drop=True)
temp_df = temp_df[~temp_df['reference_period'].isna()].reset_index(drop=True)

In [13]:
temp_df.info()
df_1 = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13560 entries, 0 to 13559
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   fips                                     13560 non-null  int64  
 1   year                                     13560 non-null  float64
 2   tele                                     13560 non-null  float64
 3   tg                                       13560 non-null  float64
 4   tot_srvcs                                13560 non-null  float64
 5   tot_srvcs_chrg                           13560 non-null  float64
 6   reference_period                         13560 non-null  object 
 7   state                                    13560 non-null  object 
 8   county                                   13560 non-null  object 
 9   state_fips                               13560 non-null  float64
 10  county_fips                              13560

In [14]:
df_1.to_csv(r'did_tele_ambulatory_nomatching_2016.csv', index=False)

##### 1. Matching for Counties that adopt telehealth beginning 2015 vs Control group

In [141]:
tg_list = tg_df[tg_df['year']>=2015].fips.values.tolist()

In [142]:
len(tg_list), len(cg_list)

(526, 2310)

In [144]:
df[df['fips'].isin(tg_list)]

,fips,year,tele
7,1003,2013,0
8,1003,2014,0
9,1003,2015,1
10,1003,2016,1
11,1003,2017,1
...,...,...,...
21849,56043,2015,0
21850,56043,2016,0
21851,56043,2017,0
21852,56043,2018,0


##### 1. Counties that started telehealth >=2015

In [49]:
cg_df = df.groupby(['fips']).sum().reset_index(level=[0])
cg_df

,fips,year,tele
0,1003,14112,5
1,1005,14112,1
2,1021,14112,2
3,1025,14112,3
4,1039,14112,3
...,...,...,...
808,56025,14112,5
809,56029,14112,1
810,56037,14112,1
811,56041,14112,3


In [45]:
df.fips.value_counts()

1003     7
39129    7
39049    7
39057    7
39061    7
        ..
23017    7
23019    7
23029    7
23031    7
56043    7
Name: fips, Length: 813, dtype: int64

In [44]:
df[df['year']==2019].sum()

fips    25408531
year     1641447
tele         513
dtype: int64

In [31]:
notele_13 = df[(df['year']==2013)&(df['tele']==0)].fips.values.tolist()
notele_14 = df[(df['year']==2014)&(df['tele']==0)].fips.values.tolist()

In [35]:
len(notele_13), len(notele_14)

(615, 566)

In [36]:
df[(df['year']==2015)&(df['fips'].isin(notele_13+notele_14))&(df['tele']==1)]

,fips,year,tele
2,1003,2015,1
23,1025,2015,1
37,1043,2015,1
79,1097,2015,1
114,4001,2015,1
...,...,...,...
5490,55087,2015,1
5497,55089,2015,1
5553,55113,2015,1
5609,55133,2015,1


In [16]:
temp_df = df[df['tele']==1].groupby(['fips']).min().reset_index(level=[0])
tg_list = temp_df[temp_df['year']==2015].fips.values.tolist()

In [28]:
temp_df = df[df['tele']==1].groupby(['fips']).min().reset_index(level=[0])
tg_list = temp_df[temp_df['year']==2014].fips.values.tolist()

In [29]:
temp_df

,fips,year,tele
0,1003,2015,1
1,1005,2014,1
2,1021,2013,1
3,1025,2015,1
4,1039,2017,1
...,...,...,...
808,56025,2013,1
809,56029,2019,1
810,56037,2014,1
811,56041,2013,1


In [27]:
len(tg_list)

92

In [17]:
tele_df_2013_2014 = tele_df[tele_df['year']<=2014]
tele_df_not_2013_2014 = tele_df[tele_df['year']>2014]

In [18]:
# These counties didn't do telehealth in 2013, 2014 but started later
tele_start_2015 = list(set(tele_df_2013_2014.county.values.tolist())-set(tele_df_not_2013_2014.county.values.tolist()))

In [106]:
print(len(tele_start_2015))
tele_start_2015_state = set(amb_df[amb_df['fips'].isin(tele_start_2015)][['fips','state','county']].drop_duplicates().state)
len(amb_df[amb_df['fips'].isin(tele_start_2015)][['fips','state','county']].drop_duplicates().state.value_counts())

38


25

- nonemergency 는 +0.17 (TG), +0.07 (CG)
- emergency 는 -0.21 (TG), +0.08 (CG)

In [104]:
amb_df[(amb_df['fips'].isin(tele_start_2015))&(amb_df['year']==2018)].pct_users_nonemeramb.mean()-\
amb_df[(amb_df['fips'].isin(tele_start_2015))&(amb_df['year']==2015)].pct_users_nonemeramb.mean()

0.17131459033979812

In [105]:
amb_df[(~amb_df['fips'].isin(tele_start_2015))&(amb_df['year']==2018)&(amb_df['state'].isin(tele_start_2015_state))].pct_users_nonemeramb.mean()-\
amb_df[(~amb_df['fips'].isin(tele_start_2015))&(amb_df['year']==2015)&(amb_df['state'].isin(tele_start_2015_state))].pct_users_nonemeramb.mean()

0.07709606079957121

##### 2. Counties that started telehealth >=2016

In [22]:
tele_df_2013_2014_2015 = tele_df[tele_df['year']<=2015]
tele_df_not_2013_2014_2015 = tele_df[tele_df['year']>2015]

In [23]:
# These counties didn't do telehealth in 2013, 2014 but started later
tele_start_2016 = list(set(tele_df_2013_2014_2015.county.values.tolist())-set(tele_df_not_2013_2014_2015.county.values.tolist()))

In [107]:
print(len(tele_start_2016))
tele_start_2016_state = set(amb_df[amb_df['fips'].isin(tele_start_2016)][['fips','state','county']].drop_duplicates().state)
len(amb_df[amb_df['fips'].isin(tele_start_2016)][['fips','state','county']].drop_duplicates().state.value_counts())

66


31

- nonemergency 는 +0.17 (TG), +0.08 (CG)
- emergency 는 -0.09 (TG), +0.15 (CG)

In [110]:
amb_df[(amb_df['fips'].isin(tele_start_2016))&(amb_df['year']==2018)].pct_users_emeramb.mean()-\
amb_df[(amb_df['fips'].isin(tele_start_2016))&(amb_df['year']==2015)].pct_users_emeramb.mean()

-0.09102865692582718

In [111]:
amb_df[(~amb_df['fips'].isin(tele_start_2016))&(amb_df['year']==2018)&(amb_df['state'].isin(tele_start_2016_state))].pct_users_emeramb.mean()-\
amb_df[(~amb_df['fips'].isin(tele_start_2016))&(amb_df['year']==2015)&(amb_df['state'].isin(tele_start_2016_state))].pct_users_emeramb.mean()

0.1597661502807295